# Assignment 6

First we have to create some SQL tables.

In [2]:
CREATE DATABASE MouseHypothalamus;

: Msg 1801, Level 16, State 3, Line 1
Database 'MouseHypothalamus' already exists. Choose a different database name.

Total execution time: 00:00:00.002

In [1]:
USE MouseHypothalamus;
DROP TABLE IF EXISTS #TempMolecules;
CREATE TABLE #TempMolecules(
    Gene_name NVARCHAR(10) NOT NULL,
    Cell_name NVARCHAR(40),
    Animal_ID int,
    Bregma float,
    Animal_sex nvarchar(6) NOT NULL,
    Behavior nvarchar(20),
    Centroid_X float NOT NULL,
    Centroid_Y float NOT NULL,
    Centroid_Z float NOT NULL,
    Total_brightness float,
    Area int,
    Error_bit tinyint,
    Error_direction tinyint
);

BULK INSERT #TempMolecules FROM '/var/data/merfish_barcodes.csv'
WITH ( 
    FIRSTROW = 2, -- skip the column headers
    --ROWS_PER_BATCH = 467052741, -- however many total rows the data has
    FIELDTERMINATOR = ',', 
    ROWTERMINATOR = '0x0a',
    KEEPNULLS
);

SELECT IDENTITY(int,1,1) as id,* INTO Molecules FROM #TempMolecules;

(467052740 rows affected)

Total execution time: 00:58:33.595

TODO: Add a column for the identity (auto-increment).

In [3]:
Use MouseHypothalamus;
SELECT IDENTITY(int,1,1) as id,* INTO Molecules FROM #TempMolecules;

: Msg 208, Level 16, State 0, Line 2
Invalid object name '#TempMolecules'.

Total execution time: 00:00:00.004

# Geometries

Let's start by creating the geometry objects for the molecules.

In [9]:
Use MouseHypothalamus;

DROP TABLE IF EXISTS MoleculesWithPoints;
SELECT 
        id, 
        Gene_name, 
        Cell_name, 
        Animal_ID, 
        Bregma, 
        Centroid_Z,
        geometry::STGeomFromText('POINT(' + CONVERT(VARCHAR(20), Centroid_X) + ' ' + CONVERT(VARCHAR(20), Centroid_Y) + ')', 0) AS point
    INTO MoleculesWithPoints 
    FROM Molecules;


(467052740 rows affected)

Total execution time: 04:10:57.025

In [2]:
Use MouseHypothalamus;
ALTER TABLE MoleculesWithPointsHead
ADD CONSTRAINT moleculesWithPointsHead_id_PK PRIMARY KEY(id);

Commands completed successfully.

Total execution time: 00:00:02.222

In [ ]:
ALTER TABLE MoleculesWithPoints
ADD CONSTRAINT moleculesWithPoints_id_PK PRIMARY KEY(id);

In [19]:
Use MouseHypothalamus;
SELECT Max(Centroid_X) as maxx, Max(Centroid_Y) as maxy,  Min(Centroid_X) as minx, Min(Centroid_Y) as miny FROM MoleculesHead;


(1 row affected)

Total execution time: 00:00:00.012

maxx,maxy,minx,miny
-2819.077,3387.196,-3017.902,3187.312


In [20]:
Use MouseHypothalamus;
SELECT Max(Centroid_X) as maxx, Max(Centroid_Y) as maxy,  Min(Centroid_X) as minx, Min(Centroid_Y) as miny FROM Molecules;

(1 row affected)

Total execution time: 00:00:03.816

maxx,maxy,minx,miny
4203.399,4968.7,-4289.699,-5009.986


In [3]:
Use MouseHypothalamus;
CREATE SPATIAL INDEX MoleculesWithPointsHeadInd ON
   [MouseHypothalamus].[dbo].[MoleculesWithPointsHead](point)
   WITH (GRIDS = (HIGH, HIGH, HIGH, HIGH), 
        BOUNDING_BOX = (XMIN = -4300,YMIN = -5100, XMAX = 4300, YMAX = 5000)); -- approx based on the actual min/max values in the data

Commands completed successfully.

Total execution time: 00:00:05.516

Now we'll create POLYGON spatial types using the CellBoundariesWithGeometryStrings data.

In [17]:
Use MouseHypothalamus;

DROP TABLE IF EXISTS CellBoundariesWithPolygonsHead;
SELECT 
        id, 
        layer,
        feature_uid,
        feature_id,
        geometry::STGeomFromText(REPLACE(geometry_string, '"', ''),0) AS polygon
    INTO CellBoundariesWithPolygonsHead
    FROM CellBoundariesHeadWithGeometryStrings;

(6930 rows affected)

Total execution time: 00:00:11.874

In [ ]:
ALTER TABLE CellBoundariesWithPolygons
ADD CONSTRAINT cellBoundariesWithPolygons_id_PK PRIMARY KEY(id);

In [4]:
ALTER TABLE CellBoundariesWithPolygonsHead
ADD CONSTRAINT cellBoundariesWithPolygonsHead_id_PK PRIMARY KEY(id);

Commands completed successfully.

Total execution time: 00:00:25.824

In [6]:
Use MouseHypothalamus;
CREATE SPATIAL INDEX CellBoundariesWithPolygonsHeadInd ON
   [MouseHypothalamus].[dbo].[CellBoundariesWithPolygonsHead](polygon)
   WITH (GRIDS = (HIGH, HIGH, HIGH, HIGH), 
        BOUNDING_BOX = (XMIN = -6000,YMIN = -6000, XMAX = 6000, YMAX = 6000)); -- rounding up to +- 6000

Commands completed successfully.

Total execution time: 00:00:07.303

# Molecule count table

Now, we can count how many molecules there are for each molecule type. We'll start by fixing our cell polygons using ST.MakeValid():

In [ ]:
USE MouseHypothalamus;
UPDATE CellBoundariesWithPolygons
    SET polygon = polygon.MakeValid();

To make our spatial queries run faster, we're going to split our molecules and polygon data into separate tables based on the z-layer everything resides in.

Now everything is ready to go for spatial queries on each layer!

In [3]:
-- Set variables to track the time 
DECLARE @startTime DATETIME
declare @endTime DATETIME
Set @starttime = getdate()

-- Find molecules in each cell for one layer
USE MouseHypothalamus;
DROP TABLE IF EXISTS MoleculesWithPointsLayer1;
SELECT mol.gene, COUNT(mol.molecule_id) as molecule_count, poly.cell, 1 as z_layer INTO MoleculesWithPointsLayer1 FROM (
    SELECT * FROM [MouseHypothalamus].[dbo].[MoleculesWithPoints] 
        WHERE z_layer=1 ) as mol
    INNER JOIN (   
        SELECT * FROM [MouseIleum].[dbo].[CellBoundariesWithPolygons] 
        WHERE z=1 ) as poly
    ON poly.polygon.STIntersects(mol.point) = 1 -- =1 is needed bc output is 0 or 1 instead of T/F
    GROUP BY mol.gene , poly.cell;

-- Set the ending time
Set @endTime = GETDATE()
SELECT @EndTime - @StartTime;

-- GO 5

: Msg 207, Level 16, State 1, Line 11
Invalid column name 'z_layer'.

Total execution time: 00:00:00.030

In [2]:
USE MouseHypothalamus;

    SELECT * FROM [MouseHypothalamus].[dbo].[MoleculesWithPoints] 
        WHERE z_layer=1 as mol
    INNER JOIN (   
        SELECT * FROM [MouseHypothalamus].[dbo].[CellPolygons] 
        WHERE z=1 ) as poly
    ON poly.polygon.STIntersects(mol.point) = 1 -- =1 is needed bc output is 0 or 1 instead of T/F

-- INSERT INTO MoleculeCountsAllLayers (molecule_id, gene, point, cell_id, cell_polygon, z_layer)
-- SELECT mol.molecule_id, mol.gene, mol.point, poly.cell as cell_id, poly.polygon as cell_polygon, 2 as z_layer INTO MoleculeCountsAllLayers FROM #Whatever



: Msg 102, Level 15, State 1, Line 3
Incorrect syntax near '('.

: Msg 156, Level 15, State 1, Line 5
Incorrect syntax near the keyword 'as'.

: Msg 156, Level 15, State 1, Line 8
Incorrect syntax near the keyword 'as'.

Total execution time: 00:00:00.030